#### [Referência principal](https://github.com/rahul-raoniar/Rahul_Raoniar_Blogs)

## Import relavant libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

## Reading dataset

In [2]:
concrete = pd.read_excel("Concrete.xlsx")
concrete.head()

,Cement,BF_slag,Fly_ash,Water,Super_plasticizer,Coarse_aggregate,Fine_aggregate,Age,Comp_str
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075


In [3]:
concrete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Cement             1030 non-null   float64
 1   BF_slag            1030 non-null   float64
 2   Fly_ash            1030 non-null   float64
 3   Water              1030 non-null   float64
 4   Super_plasticizer  1030 non-null   float64
 5   Coarse_aggregate   1030 non-null   float64
 6   Fine_aggregate     1030 non-null   float64
 7   Age                1030 non-null   int64  
 8   Comp_str           1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.5 KB


In [4]:
concrete.describe()

,Cement,BF_slag,Fly_ash,Water,Super_plasticizer,Coarse_aggregate,Fine_aggregate,Age,Comp_str
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.165631,73.895485,54.187136,181.566359,6.203112,972.918592,773.578883,45.662136,35.817836
std,104.507142,86.279104,63.996469,21.355567,5.973492,77.753818,80.175427,63.169912,16.705679
min,102.000000,0.000000,0.000000,121.750000,0.000000,801.000000,594.000000,1.000000,2.331808
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.707115
50%,272.900000,22.000000,0.000000,185.000000,6.350000,968.000000,779.510000,28.000000,34.442774
75%,350.000000,142.950000,118.270000,192.000000,10.160000,1029.400000,824.000000,56.000000,46.136287
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.599225


## Train test split

In [5]:
X = concrete.drop("Comp_str", axis = 1)
y = concrete["Comp_str"]

print(X.shape)
print(y.shape)

(1030, 8)
(1030,)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [7]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Grid search

## Preparing grid

In [8]:
# Create a Random Forest Classifier with specified criterion
gb_regressor = GradientBoostingRegressor(criterion = "mae")

# Create the parameter grid
param_grid = {'max_depth' : [2, 4, 8, 10, 12],
              'n_estimators' : [100, 200, 300],
              'max_features' : ['auto', 'sqrt'],
              "criterion" : ["friedman_mse", "mse", "mae"]} 

# Create a GridSearchCV object
grid_gb = GridSearchCV(
    estimator = gb_regressor,
    param_grid = param_grid,
    scoring = 'neg_mean_absolute_error',
    n_jobs = 4,
    cv = 10,
    refit = True,
    return_train_score = True)

print(grid_gb)

GridSearchCV(cv=10, estimator=GradientBoostingRegressor(criterion='mae'),
             n_jobs=4,
             param_grid={'criterion': ['friedman_mse', 'mse', 'mae'],
                         'max_depth': [2, 4, 8, 10, 12],
                         'max_features': ['auto', 'sqrt'],
                         'n_estimators': [100, 200, 300]},
             return_train_score=True, scoring='neg_mean_absolute_error')


## Train model

In [9]:
grid_gb.fit(X_train_scaled, y_train)

GridSearchCV(cv=10, estimator=GradientBoostingRegressor(criterion='mae'),
             n_jobs=4,
             param_grid={'criterion': ['friedman_mse', 'mse', 'mae'],
                         'max_depth': [2, 4, 8, 10, 12],
                         'max_features': ['auto', 'sqrt'],
                         'n_estimators': [100, 200, 300]},
             return_train_score=True, scoring='neg_mean_absolute_error')

### Best parameters

In [10]:
grid_gb.best_params_

{'criterion': 'mse',
 'max_depth': 4,
 'max_features': 'sqrt',
 'n_estimators': 300}

### Best score

In [11]:
grid_gb.best_score_

-2.782602145634087

In [12]:
cv_results = pd.DataFrame(grid_gb.cv_results_)

# Extract and print the row that had the best mean test score
best_row = cv_results[cv_results['rank_test_score'] == 1]
print(best_row)

    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
41       0.364101      0.031693         0.002497        0.000921   

   param_criterion param_max_depth param_max_features param_n_estimators  \
41             mse               4               sqrt                300   

                                               params  split0_test_score  ...  \
41  {'criterion': 'mse', 'max_depth': 4, 'max_feat...          -2.909463  ...   

    split2_train_score  split3_train_score  split4_train_score  \
41           -1.151883           -1.112935           -1.176433   

    split5_train_score  split6_train_score  split7_train_score  \
41            -1.17894           -1.191538           -1.139731   

    split8_train_score  split9_train_score  mean_train_score  std_train_score  
41           -1.211992           -1.190432         -1.176775         0.035209  

[1 rows x 34 columns]


In [13]:
cv_results.loc[[grid_gb.best_index_]]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_max_features,param_n_estimators,params,split0_test_score,...,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,split8_train_score,split9_train_score,mean_train_score,std_train_score
41,0.364101,0.031693,0.002497,0.000921,mse,4,sqrt,300,"{'criterion': 'mse', 'max_depth': 4, 'max_feat...",-2.909463,...,-1.151883,-1.112935,-1.176433,-1.17894,-1.191538,-1.139731,-1.211992,-1.190432,-1.176775,0.035209


In [14]:
grid_gb.best_params_["n_estimators"]

300

## Predict test score

In [15]:
predictions = grid_gb.best_estimator_.predict(X_test_scaled)

In [16]:
mean_absolute_error(predictions, y_test)

2.795385910770736

# Random Search

In [17]:
# Create a Random Forest Classifier with specified criterion
gb_regressor = GradientBoostingRegressor(criterion = "mae")

# Create the parameter grid
param_grid = {'max_depth' : [2, 4, 8, 10, 12],
              'n_estimators' : [100, 200, 300],
              'max_features' : ['auto', 'sqrt'],
              "criterion" : ["friedman_mse", "mse", "mae"]} 

# Create a GridSearchCV object
random_grid_gb = RandomizedSearchCV(
    estimator = gb_regressor,
    param_distributions = param_grid,
    scoring = 'neg_mean_absolute_error',
    n_jobs = 4,
    cv = 10,
    refit = True, return_train_score = True)
print(random_grid_gb)

RandomizedSearchCV(cv=10, estimator=GradientBoostingRegressor(criterion='mae'),
                   n_jobs=4,
                   param_distributions={'criterion': ['friedman_mse', 'mse',
                                                      'mae'],
                                        'max_depth': [2, 4, 8, 10, 12],
                                        'max_features': ['auto', 'sqrt'],
                                        'n_estimators': [100, 200, 300]},
                   return_train_score=True, scoring='neg_mean_absolute_error')


In [18]:
random_grid_gb.fit(X_train_scaled, y_train)

RandomizedSearchCV(cv=10, estimator=GradientBoostingRegressor(criterion='mae'),
                   n_jobs=4,
                   param_distributions={'criterion': ['friedman_mse', 'mse',
                                                      'mae'],
                                        'max_depth': [2, 4, 8, 10, 12],
                                        'max_features': ['auto', 'sqrt'],
                                        'n_estimators': [100, 200, 300]},
                   return_train_score=True, scoring='neg_mean_absolute_error')

In [19]:
print(random_grid_gb.cv_results_['param_max_depth'])
print(random_grid_gb.cv_results_['param_n_estimators'])

[12 10 2 4 4 2 4 8 2 8]
[100 200 300 200 200 100 100 100 100 100]


In [20]:
random_grid_gb.best_params_

{'n_estimators': 100,
 'max_features': 'sqrt',
 'max_depth': 8,
 'criterion': 'mse'}

In [21]:
random_grid_gb.best_score_

-3.009400472938249

In [22]:
predictions = random_grid_gb.best_estimator_.predict(X_test_scaled)

In [23]:
mean_absolute_error(predictions, y_test)

3.085978632092046

## Genetic hyperparameters tuning

In [27]:
from tpot import TPOTRegressor

In [28]:
# Assign the values outlined to the inputs
number_generations = 20
population_size = 10
offspring_size = 10
scoring_function = 'neg_mean_absolute_error'

# Create the tpot classifier
tpot_clf = TPOTRegressor(generations = number_generations,
                         population_size = population_size,
                         offspring_size = offspring_size,
                         scoring = scoring_function,
                         verbosity = 2,
                         random_state = 2,
                         cv = 10,
                         n_jobs = -1)

# Fit the classifier to the training data
tpot_clf.fit(X_train_scaled, y_train)

# Score on the test set
print(tpot_clf.score(X_test_scaled, y_test))

Optimization Progress:   0%|          | 0/210 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -3.028483254521133

Generation 2 - Current best internal CV score: -3.0232138198951146

Generation 3 - Current best internal CV score: -3.0232138198951146

Generation 4 - Current best internal CV score: -3.0232138198951146

Generation 5 - Current best internal CV score: -3.0232138198951146

Generation 6 - Current best internal CV score: -3.0232138198951146

Generation 7 - Current best internal CV score: -3.0232138198951146

Generation 8 - Current best internal CV score: -2.9996221125186024

Generation 9 - Current best internal CV score: -2.9996221125186024

Generation 10 - Current best internal CV score: -2.9828855398193816

Generation 11 - Current best internal CV score: -2.9828855398193816

Generation 12 - Current best internal CV score: -2.9342178716345044

Generation 13 - Current best internal CV score: -2.9342178716345044

Generation 14 - Current best internal CV score: -2.9325544289876646

Generation 15 - Current best internal CV sc

In [36]:
predictions = tpot_clf.predict(X_test_scaled)

In [40]:
y_test[0:1]

31    52.90832
Name: Comp_str, dtype: float64

In [41]:
predictions[0:1]

array([51.695007], dtype=float32)

In [ ]:
pd